In [8]:
import pickle
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [9]:
directory = '/mnt/home/jbielecki1/NEMA/'
modelFileName = '10000000/ADA/adaEstimators1000Depth6'
max_depth = 6

In [10]:
model = pickle.load(open(directory + modelFileName, 'rb'))
data = pickle.load(open(directory + 'NEMA_IQ_384str_N0_1000_COINCIDENCES_PREPARED_part01', 'rb'))
data2 = pickle.load(open(directory + 'NEMA_IQ_384str_N0_1000_COINCIDENCES_PREPARED_part02', 'rb'))
data3 = pickle.load(open(directory + 'NEMA_IQ_384str_N0_1000_COINCIDENCES_PREPARED_part03', 'rb'))
allData = pd.concat([data, data2, data3])

In [11]:
codes = {1:1, 2:0, 3:0, 4:0}
y = allData["class"].map(codes)
x = allData.drop(["t1", "t2", "sX1", "sY1", "sZ1", "class", "rError"], axis = 1)

In [12]:
print("Percetage of positive probes: " + str(len(y[y == 1])/len(y) * 100) + "%")

Percetage of positive probes: 29.172603333333335%


In [ ]:
test_accuracy = []
max_acc = 0
y_pred_prob = []
y_pred = []

for test_predicts_el in model.staged_predict_proba(x):
    y_pred_prob_el = test_predicts_el[:,1]
    y_pred_el = y_pred_prob_el > 0.5
    acc = accuracy_score(y_pred_el, np.array(y))
    
    if acc > max_acc:
        max_acc = acc
        y_pred_prob = y_pred_prob_el
        y_pred = y_pred_el
        
    test_accuracy.append(acc)

In [ ]:
bestAccuracy = max(test_accuracy)
bestNEstimators = test_accuracy.index(max(test_accuracy))

In [8]:
print(bestAccuracy)

0.8850935


In [9]:
pPsOrginalPositive = x[y > 0]
pPsOrginalNegative = x[y == 0]
pPsPredictedPositive = x[y_pred]
pPsPredictedNegative = x[y_pred == 0]

In [13]:
dataPositive = data.iloc[list(pPsPredictedPositive.index),:]

In [14]:
dataRec = dataPositive.iloc[:,:16]

In [15]:
len(dataRec)

3812932

In [16]:
dataRec.to_csv(directory + 'adaReconstruction', sep = "\t", header = False, index = False)